# Incremental Proper Orthogonality Decomposition based Reduced Order Modeling (iPOD-ROM)

In this notebook, we demonstrate how the incremental Proper Orthogonality Decomposition (iPOD) can be used for reduced order modeling. This gives us a barebones implementation of the [MORe DWR paper](https://doi.org/10.48550/arXiv.2304.01140) by Fischer et al. The major difference to this publication is that here we will not use space-time finite elements and goal-oriented adaptivity (dual-weighted residual method). This simplifies the problem and we focus only on the incremental reduced order modeling. For practical applications, one should use some a posteriori estimates, e.g. [dual-weighted residual error estimates](https://ganymed.math.uni-heidelberg.de/Paper/Preprint2001-03.pdf), but here we use the true error as an indicator to switch between the full order model (FOM) and the reduced order model (ROM). Obviously the true error is not known in general. 

In [ ]:
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

In [ ]:
from fenics import *
import numpy as np
import matplotlib.pyplot as plt
import scipy
import math

PLOT = False

## Routine for incremental Proper Orthogonal Decomposition (iPOD)

In [ ]:
def iPOD(
    POD,  # POD basis
    snapshot,  # new snapshot to be added to POD basis
    bunch_matrix,  # bunch matrix
    bunch_size,  # desired size of bunch matrix
    singular_values,  # singular values of POD basis
    total_energy,  # total energy of POD basis
    energy_content,  # desired energy content
):

    if bunch_matrix.shape[1] == 0:
        # initialize bunch matrix if empty
        bunch_matrix = snapshot.reshape(-1, 1)  # np.empty([np.shape(snapshot)[0], 0])
    else:
        # concatenate new snapshot to bunch matrix
        bunch_matrix = np.hstack((bunch_matrix, snapshot.reshape(-1, 1)))

    # add energy of new snapshot to total energy
    total_energy += np.dot((snapshot), (snapshot))

    # check bunch_matrix size to decide if to update POD
    if bunch_matrix.shape[1] == bunch_size:
        # initialize POD with first bunch matrix
        if POD.shape[1] == 0:
            POD, S, _ = scipy.linalg.svd(bunch_matrix, full_matrices=False)

            # compute the number of POD modes to be kept
            r = 0
            while (np.dot(S[0:r], S[0:r]) / total_energy <= energy_content) and (
                r <= np.shape(S)[0]
            ):
                r += 1

            singular_values = S[0:r]
            POD = POD[:, 0:r]
        # update POD with  bunch matrix
        else:
            M = np.dot(POD.T, bunch_matrix)
            P = bunch_matrix - np.dot(POD, M)

            Q_p, R_p = scipy.linalg.qr(P, mode="economic")
            Q_q = np.hstack((POD, Q_p))

            S0 = np.vstack(
                (
                    np.diag(singular_values),
                    np.zeros((np.shape(R_p)[0], np.shape(singular_values)[0])),
                )
            )
            MR_p = np.vstack((M, R_p))
            K = np.hstack((S0, MR_p))

            # check the orthogonality of Q_q heuristically
            if np.inner(Q_q[:, 0], Q_q[:, -1]) >= 1e-10:
                Q_q, R_q = scipy.linalg.qr(Q_q, mode="economic")
                K = np.matmul(R_q, K)

            # inner SVD of K
            U_k, S_k, _ = scipy.linalg.svd(K, full_matrices=False)

            # compute the number of POD modes to be kept
            r = 0 
            while (np.dot(S_k[0:r], S_k[0:r]) / total_energy <= energy_content) and (
                r < np.shape(S_k)[0]
            ):
                r += 1

            singular_values = S_k[0:r]
            POD = np.matmul(Q_q, U_k[:, 0:r])

        # empty bunch matrix after update
        bunch_matrix = np.empty([np.shape(bunch_matrix)[0], 0])

    return POD, bunch_matrix, singular_values, total_energy


## Solving the Full Order Model

As a numerical expample, we consider a heat equation on a square with a rotating heat source. This problem has been taken from Section 5.2 from the [MORe DWR paper](https://doi.org/10.48550/arXiv.2304.01140). 

The strong formulation of the heat equation reads: \\
Find $u: [0,T] \times \bar\Omega \rightarrow \mathbb{R}$ such that
$$
\partial_t u  - \Delta_x u = f \quad \text{in} \quad (0,T) \times \Omega, \\
\qquad\qquad\quad  u = 0 \quad \text{on} \quad (0,T) \times \partial\Omega, \\
\qquad\qquad  u = u_0 \quad \text{on} \quad \lbrace 0 \rbrace \times \Omega,
$$
where $\Omega \subset \mathbb{R}^d$ with $d \in \lbrace 1, 2, 3 \rbrace$ is an open domain and the Laplacian of $u$, denoted by $\Delta_x u$, is defined as
$$
\Delta_x u = \sum_{i = 1}^d \partial_{x_i} u.
$$

### Temporal discretization: Backward Euler method
To solve this time dependent PDE, we first discretize it in time with finite differences and then use finite elements in space. For the temporal discretization, we use the backward Euler method, which reads for the heat equation
$$
\frac{u^{n+1} - u^n}{\Delta t} - \Delta_x u^{n+1} = f^{n+1}.
$$


Here, we denote the constant time step size as $\Delta t := t_{n+1}-t_n$ and use the short hand notation $u^{n+1} := u(t_{n+1})$ and $u^n := u(t_n)$.

### Variational formulation of heat equation
Now that the heat equation has already been discretized in time, we discretize it in space with finite elements. For this we introduce the space $V := H^1_0(\Omega)$ as the space of functions that vanish at $\partial \Omega$ whose weakly derivattives are $L^2(\Omega)$-integrable. After testing with a function $v \in V$, we get
$$
\int_\Omega \left( u^{n+1} - \Delta t \cdot \Delta_x u^{n+1} \right) \cdot v\ \mathrm{d}(x,y)
= 
\int_\Omega \left(
u^n + \Delta t \cdot f^{n+1}
\right) \cdot v\ \mathrm{d}(x,y).
$$
Applying integration by parts, the variational formulation of the heat equation thus reads: \\
Find $u^{n+1} \in V$ such that
$$
a(u^{n+1}, v) = l(v) \qquad \forall v \in V,
$$
where
$$
a(u, v) := \int_\Omega u \cdot v\ \mathrm{d}(x,y) + \Delta t \cdot \int_\Omega \nabla_x u \cdot \nabla_x v\ \mathrm{d}(x,y), \tag{1}
$$
$$
l(v) := \int_\Omega u^n \cdot v\ \mathrm{d}(x,y) +  \Delta t \cdot \int_\Omega f^{n+1} \cdot v\ \mathrm{d}(x,y) . \tag{2}
$$
<br>

Using the mass matrix $M \in \mathbb{R}^{m \times m}$ with $m := \text{dim}(V)$, which is defined by
$$
M_{ij} = \int_\Omega \varphi_i \cdot \varphi_j\ \mathrm{d}(x,y), 
$$
and the Laplacian/stiffness matrix $K \in \mathbb{R}^{m \times m}$, which is defined by
$$
K_{ij} = \int_\Omega \nabla_x \varphi_i \cdot \nabla_x \varphi_j\ \mathrm{d}(x,y),
$$
the linear equation system reads: \\
Find $\vec{U}^{n+1} \in \mathbb{R}^m$ such that
$$
[M + \Delta t \cdot K]\vec{U}^{n+1} = M\vec{U}^n + \Delta t \cdot \vec{F}^{n+1}. 
$$

### Model problem for the heat equation
Using the model problem from Section 5.2 from the [MORe DWR paper](https://doi.org/10.48550/arXiv.2304.01140), we have the domain $\Omega = (0,1) \times (0,1)$, homogeneous Dirichlet boundary conditions and initial conditions
$$
u(0,x) = 0 \qquad \forall x \in \Omega, \\
u(t,x) = 0 \qquad \forall x \in \partial \Omega,
$$
and the right hand side function
$$
f(t, x) := \begin{cases}
        \sin(4 \pi t)  & \text{if } (x_1 - p_1)^2 + (x_2 - p_2)^2 < r^2,\\
        0 & \text{else},
    \end{cases}
$$
with $x = (x_1, x_2)$, midpoint $p = (p_1, p_2) = (\frac{1}{2}+\frac{1}{4} \cos(2 \pi t), \frac{1}{2}+\frac{1}{4} \sin(2 \pi t))$ and radius of the trajectory $r=0.125$.

In [ ]:
# Source: https://github.com/mathmerizing/POD-ROM/blob/main/POD_Fenics.ipynb

# start time 
t = 0.
# end time 
T = 5.
# time step size
Δt = 0.01
     
# defining the mesh
nx = ny = 50
mesh = UnitSquareMesh(nx, ny)
# visualize the triangulation
if PLOT:
  plot(mesh)
  plt.show()
     
V = FunctionSpace(mesh, 'P', 1)
bc = DirichletBC(V, Constant(0.), lambda _, on_boundary: on_boundary)

# initial condition
u_0 = Constant(0.)
# u_n: solution from last time step
u_n = interpolate(u_0, V)
     
# Define variational problem
u = TrialFunction(V)
v = TestFunction(V)

# mass matrix
M = assemble(u*v*dx)

# Laplace matrix
K = assemble(dot(grad(u), grad(v))*dx)

# right hand side function
class RHSExpression(UserExpression):
    _t = 0.
    def set_time(self, t):
      self._t = t

    def eval_cell(self, value, x, ufc_cell):
      if ((x[0] - 0.5 - 0.25 * np.cos(2. * np.pi * self._t))**2 + (x[1] - 0.5 - 0.25 * np.sin(2. * np.pi * self._t))**2 < 0.125**2):
        value[0] = np.sin(4. * np.pi * self._t)
      else:
        value[0] = 0.
    
    def value_shape(self):
        return () # scalar function

rhs_func = RHSExpression()

# system matrix
system_matrix = M + Δt * K

# right hand side matrix
rhs_matrix = M
     
# number of snapshots
n = int(math.ceil((T-t)/Δt))+1

# number of degrees of freedom
m = V.dim()

# snapshot matrix
Y = np.zeros((m, n))

# store initial condition in snapshot matrix
Y[:, 0] = u_n.vector() 

# Time-stepping
u = Function(V) # u_{n+1}: current solution

n = 1
while(t+Δt <= T+1e-8):
    # Update current time
    t += Δt

    # Compute solution
    A = system_matrix
    # Compute RHS
    b = u_n.vector().copy()
    rhs_matrix.mult(u_n.vector(), b)
    # Add force terms
    rhs_func.set_time(t)
    b += assemble(rhs_func*v*dx)
    # Apply boundary conditions
    bc.apply(A,b)
    # Solve linear system
    solve(A, u.vector(), b)

    # Plot solution
    if PLOT:
      plt.title(f"t = {round(t,4)}")
      c = plot(u)
      plt.colorbar(c)
      plt.show()

    # Store solution in snapshot matrix
    Y[:, n] = u.vector()
    n += 1
    
    # Update previous solution
    u_n.assign(u)

## Solving the incremental Reduced Order Model
To make the computations cheaper, we now want to replace the large linear equation system from the finite element simulations with a reduced linear system of size $r \ll m$. A popular approach for this is Proper Orthogonal Decomposition (POD) based reduced order modeling (ROM), where one tries to find a new basis from the snapshots of the high fidelity simulation and then carries out all remaining simulations in this low dimensional function space. A mathematically rigorous overview of this topic can be found in <a href="http://www.math.uni-konstanz.de/numerik/personen/volkwein/teaching/POD-Book.pdf">Prof. Volkwein's lecture notes</a> or in [this GitHub repository](https://github.com/mathmerizing/POD-ROM). Here, we now do things slightly differently and update the POD basis on-the-fly. This has been described in Section 4.2.1 of the [MORe DWR paper](https://doi.org/10.48550/arXiv.2304.01140).

In [ ]:
# change from the FOM to the POD basis
def reduce_matrix(matrix, pod_basis):
  return np.dot(np.dot(pod_basis.T, matrix.array()), pod_basis)

def reduce_vector(vector, pod_basis):
  return np.dot(pod_basis.T, np.array(vector))

# change from POD to FOM basis
def project_vector(vector, pod_basis):
  return np.dot(pod_basis, vector)

# Initialize POD setting
POD = np.empty([0,0])
singular_values = np.empty([0])
total_energy = 0.
bunch_matrix = np.empty([0,0])
BUNCH_SIZE = 1
ENERGY_CONTENT = 0.999999

# create an initial POD basis from the first FOM snapshot
POD, bunch_matrix, singular_values, total_energy = iPOD(\
      POD, Y[:, 1], bunch_matrix, BUNCH_SIZE, \
      singular_values, total_energy, ENERGY_CONTENT)


reduced_system_matrix = reduce_matrix(system_matrix, POD)
reduced_rhs_matrix = reduce_matrix(rhs_matrix, POD)

# start time 
t = 0.

# u_n: solution from last time step
u_n = interpolate(u_0, V)
u_n_ROM = reduce_vector(u_n.vector(), POD)

# number of snapshots
n = int(math.ceil((T-t)/Δt))+1

# snapshot matrix
Y_ROM = np.zeros((m, n))

# store initial condition in snapshot matrix
Y_ROM[:, 0] = u_n.vector() 

# Time-stepping
n = 1
FOM_solves = 0
record_basis_size = []

while(t+Δt <= T+1e-8):
    # Update current time
    t += Δt

    # 1. Solve timestep with ROM
    rhs_func.set_time(t)

    u_n_ROM = np.linalg.solve(
        reduced_system_matrix, 
        np.dot(reduced_rhs_matrix, u_n_ROM) + reduce_vector(assemble(rhs_func*v*dx), POD)
    )

    # Prolongate the reduced solution into the FOM space
    u.vector()[:] = project_vector(u_n_ROM, POD)

    # 2. Check whether ROM solution is good enough with error estimator
    # NOTE: Here we use the true error to simplify the code, but this would be impractical for real applications.
    # check norm of error between ROM solution and FOM solution
    error = np.linalg.norm(u.vector()[:] - Y[:, n])
    error_rel = np.linalg.norm(u.vector()[:] - Y[:, n])/np.linalg.norm(Y[:, n])

    # 3. If error from Step 2 is too large: Solve timestep with FOM and update POD basis
    if error_rel > 1e-2:
      print(f"  INFO: Running FOM for t = {round(t,4)} with relative error: {error_rel * 100} %")
      FOM_solves += 1
      # Solve timestep with FOM
      A = system_matrix
      # Compute RHS
      b = u_n.vector().copy()
      rhs_matrix.mult(u_n.vector(), b)
      # Add force terms
      b += assemble(rhs_func*v*dx)
      # Apply boundary conditions
      bc.apply(A,b)
      # Solve linear system
      solve(A, u.vector(), b)

      # update POD basis
      POD, bunch_matrix, singular_values, total_energy = iPOD(\
        POD, u.vector().get_local(), bunch_matrix, BUNCH_SIZE, \
        singular_values, total_energy, ENERGY_CONTENT)

      # update reduced matrices
      reduced_system_matrix = reduce_matrix(system_matrix, POD)
      reduced_rhs_matrix = reduce_matrix(rhs_matrix, POD)

      # update old solution to new basis
      u_n_ROM = reduce_vector(u.vector(), POD) 

    # Plot solution
    if PLOT:
      plt.title(f"t = {round(t,4)}")
      c = plot(u)
      plt.colorbar(c)
      plt.show()

    # Store solution in snapshot matrix
    Y_ROM[:, n] = u.vector()
    n += 1
    record_basis_size.append(POD.shape[1])
    # Update previous solution
    u_n.assign(u)

print(f"FOM solves / total solves: {FOM_solves} / {n}")

In [ ]:
plt.plot(np.linspace(0, T, n-1), record_basis_size)
plt.grid()
plt.xlabel("$t$ $[$s$]$",fontsize=16)
plt.ylabel("POD basis size",fontsize=16)
plt.show()

temporal_error = []
for i in range(n):
  temporal_error.append(100*np.linalg.norm(Y_ROM[:, i] - Y[:, i])/np.linalg.norm(Y[:, i]))

plt.plot(np.linspace(0, T, n), temporal_error)
plt.grid()
plt.xlabel("$t$ $[$s$]$",fontsize=16)
plt.ylabel("Relative error [%]",fontsize=16)
plt.show()

In [ ]:
# Plotting 
from matplotlib.animation import FuncAnimation
from matplotlib.ticker import MaxNLocator, FormatStrFormatter
from IPython.display import HTML, display
import os

if not os.path.exists("images"):
    os.makedirs("images")

u_FOM = Function(V)
u_ROM = Function(V)
u_diff = Function(V)

u_max = np.max(np.max(Y))
u_min = np.min(np.min(Y))

diff_max = np.max(np.max(Y - Y_ROM))
diff_min = np.min(np.min(Y - Y_ROM))

size_colorbar = 0.9
pad_space = 0.2
skip_frames = 5
for i in range(0,n,skip_frames):
  fig, axes = plt.subplots(1, 3, figsize=(10, 4))  

  u_FOM.vector().set_local(Y[:, i])
  u_ROM.vector().set_local(Y_ROM[:, i])
  u_diff.vector().set_local(Y[:, i] - Y_ROM[:, i])

  plt.subplot(1, 3, 1)
  c = plot(u_FOM, title='Velocity - FOM', vmin=u_min, vmax=u_max)
  cbar = plt.colorbar(c, orientation='horizontal', ticks=MaxNLocator(nbins=3), format=FormatStrFormatter('%.2f'), pad=pad_space, shrink=size_colorbar)
  cbar.mappable.set_clim(u_min, u_max)

  plt.subplot(1, 3, 2)
  c = plot(u_ROM, title='Velocity - ROM', vmin=u_min, vmax=u_max)
  cbar = plt.colorbar(c, orientation='horizontal', ticks=MaxNLocator(nbins=3), format=FormatStrFormatter('%.2f'), pad=pad_space, shrink=size_colorbar)
  cbar.mappable.set_clim(u_min, u_max)

  plt.subplot(1, 3, 3)
  c = plot(u_diff, title='Velocity - Difference', vmin=diff_min, vmax=diff_max)
  cbar = plt.colorbar(c, orientation='horizontal', ticks=MaxNLocator(nbins=3), format=FormatStrFormatter('%.4f'), pad=pad_space, shrink=size_colorbar)
  cbar.mappable.set_clim(diff_min, diff_max)
  
  plt.tight_layout()

  plt.savefig(f"images/solution{i:05}.png")
  plt.clf()
  plt.close(fig)
  
# generate video
if os.path.exists("out.mp4"):
    os.remove("out.mp4")

!ffmpeg -framerate 10 -pattern_type glob -i 'images/*.png' -c:v libx264 -r 30 -pix_fmt yuv420p out.mp4

# play video
from base64 import b64encode
mp4 = open('out.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=1500 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)